In [4]:
import os
#It goes back to the previous folder (main folder) as we're in research folder
os.chdir('../')

In [5]:
print(os.getcwd())

d:\Data Science\Python Assignment\End to End Kidney Disease Detection


Update the entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

#returning all config parameters which will be used in arrow function output entities
#@dataclass(frozen=True) is a decorator in Python's dataclasses module that makes instances of the class immutable after creation. 
# When applied, it provides the following benefits:
# Immutability: Once an obSject is created, you cannot change its attributes. This is enforced by raising an error if there is any attempt to modify the instance's fields.
# Hashability: Instances of the class become hashable (i.e., you can use them as keys in dictionaries or add them to sets) as long as all their fields are hashable.
@dataclass(frozen=True)
class PrepareBaseMLConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


Update the configuration manager in src config

In [10]:
from DiseaseClassifier.constants import *
from DiseaseClassifier.utils.common import read_yaml, create_directories


    
class ConfigurationManager:
    def __init__(self,
                 #from constants
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        #init file for referencing the config and params files
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        #we have artifacts_root in config file
        #It'll go through directories and create artifacts folder and subfolders
        create_directories([self.config.artifacts_root])

    #This method creates the directory structure needed for the base machine learning configuration 
    #and returns an instance of PrepareBaseMLConfig
    #we've defined DataIngestionConfig class above
    def get_prepare_base_ml_config(self)->PrepareBaseMLConfig:
        #prepare_base_ML in artifacts, config file 
        config=self.config.prepare_base_ML
        #referring to prepare_base_ML, artifacts, config file
        create_directories([config.root_dir])

        prepare_base_ml_config=PrepareBaseMLConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
            )
        return prepare_base_ml_config


In [8]:
ConfigurationManager().get_prepare_base_ml_config().params_image_size

[2024-10-10 18:50:12,667: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-10-10 18:50:12,709: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-10 18:50:12,712: INFO: common: created directory at: <class 'pathlib.Path'>]
[2024-10-10 18:50:12,795: INFO: common: created directory at: <class 'pathlib.Path'>]


BoxList([224, 224, 3])

In [15]:
from DiseaseClassifier.constants import CONFIG_FILE_PATH
print(CONFIG_FILE_PATH)

config\config.yaml


In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

#downloading the pre-trained vgg16
#In PrepareBaseModel, self.config is an instance of PrepareBaseMLConfig, which contains the model parameters such as params_image_size, params_weights, etc.
class PrepareBaseModel:
    #look at the parameters
    def __init__(self,config:PrepareBaseMLConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            #The reason you're using self.config is because you are passing an instance of PrepareBaseMLConfig into the PrepareBaseModel class. 
            # This configuration object contains all the necessary parameters that were extracted from the params.yaml file (e.g., IMAGE_SIZE, WEIGHTS, INCLUDE_TOP).
            # If you wanted to use self.params, you'd need to ensure that params is accessible in this context. 
            # But since self.config contains the params from the configuration, it's appropriate to use self.config.params_image_size etc., within the PrepareBaseModel class.
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path,model=self.model)
    
    #Why Use @staticmethod?
    # No Need for self: If the method doesn't need to access or modify the instance's attributes or methods, it’s better to make it a static method.
    # Grouping Related Functions: Sometimes, it’s just about logically grouping functions that belong together. Even if the function doesn't use instance variables, it might conceptually belong to the class.

    #This method keep some layers of the pre-trained model
    @staticmethod
    def prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.tarinable=False
        elif (freeze_till is not None) and (freeze_till>0):
            #This line freezes all layers except the last freeze_till layers. Here's how it works:
            # The slice model.layers[:-freeze_till] grabs all layers except the last freeze_till layers (that's why you have the negative sign).
            # For example, if freeze_till = 5, this means the last 5 layers will remain trainable, while the rest will be frozen.
            for layer in model.layers[:-freeze_till]:
                layer.trainable=False

        #The convolutional layers in models like VGG16 output a 3D tensor (height, width, and channels), but fully connected layers (like Dense) expect a 1D vector. 
        #The Flatten layer is used to reshape that 3D tensor into a single vector, which can be fed into a Dense layer.
        flatten_in=tf.keras.layers.Flatten()(model.output)
        #creating the custom dense layer which we didn't freeze
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(flatten_in)

        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

        full_model.compile(
            optimizer=optimizer,
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy','recall']
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model=self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    

Creating the data pipeline

In [11]:
try:
    config= ConfigurationManager()
    
    prepare_base_model_config=config.get_prepare_base_ml_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2024-10-27 22:14:24,647: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-27 22:14:24,651: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-27 22:14:24,654: INFO: common: created directory at: <class 'pathlib.Path'>]
[2024-10-27 22:14:24,659: INFO: common: created directory at: <class 'pathlib.Path'>]
[2024-10-27 22:14:26,781: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 14,764,866 (56.32 MB)

 Non-trainable params: 0 (0.00 B)